In [1]:
%load_ext autoreload
%autoreload 1
%aimport prepare_data

from prepare_data import * 
from pygenesig.tools import collapse_matrix
from pygenesig.file_formats import * 

import numpy as np
import pandas as pd

# Preprocess mouseGNF
For the crossvalidation we need a
* matrix of gene expression data
* list of target classes (signatures) 

## Load data

In [2]:
!ls ../../data/mouseGNF/

map_orthologoues.tsv	  mouseGNF-phenoData.txt     mouse.txt
mouseGNF-featureData.txt  mouseGNF-signalMatrix.gct


In [3]:
hgnc_symbols = pd.read_csv("../../data/hgnc_complete_set.txt", sep="\t").symbol.tolist()

/pstore/apps/Anaconda3/4.1.1/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (32,34,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
pdata = pd.read_csv("../../data/mouseGNF/mouseGNF-phenoData.txt", sep="\t", index_col=0)

In [5]:
gct = pd.read_csv("../../data/mouseGNF/mouseGNF-signalMatrix.gct", sep="\t", skiprows=2, index_col=0)

In [11]:
human_to_mouse = pd.read_csv("../../data/mouseGNF/map_orthologoues.tsv", delimiter="\t")

In [13]:
human_to_mouse.shape, gct.shape, pdata.shape

((45101, 5), (45101, 183), (182, 33))

In [27]:
target_series = pdata.loc[~pdata["GEO.source.name"].isnull(), "GEO.source.name"].apply(
    lambda x: "_".join(x.split("_")[:-1]))

In [38]:
exprs = gct.loc[:, target_series.index].values

In [32]:
target = target_series.as_matrix()

In [35]:
gene_symbols = human_to_mouse.GeneSymbol

In [36]:
expr.shape, target.shape, gene_symbols.shape

((45101, 160), (160,), (45101,))

### Collapse gene expression by symbol

In [39]:
exprs_collapsed = collapse_matrix(exprs, gene_symbols.tolist(), axis=0, aggregate_fun=np.sum)

In [40]:
exprs_collapsed.shape

(16020, 160)

In [41]:
exprs_collapsed = exprs_collapsed[exprs_collapsed.index.isin(hgnc_symbols)]

In [42]:
exprs_collapsed.shape

(15851, 160)

## Store results

In [43]:
!mkdir -p ../../data_processed/mouseGNF

In [45]:
write_target(target, "../../data_processed/mouseGNF/target.csv")
write_rosetta(exprs_collapsed.index.tolist(), "../../data_processed/mouseGNF/rosetta.csv")

In [46]:
write_expr(exprs_collapsed.as_matrix(), "../../data_processed/mouseGNF/exprs.npy")

In [47]:
np.unique(target)

array(['B-cells_marginal_zone', 'NK_cells', 'T-cells_CD4+', 'T-cells_CD8+',
       'T-cells_foxP3+', 'adipose_brown', 'adipose_white', 'adrenal_gland',
       'amygdala', 'bladder', 'bone', 'bone_marrow', 'cerebellum',
       'cerebral_cortex', 'cerebral_cortex_prefrontal', 'ciliary_bodies',
       'common_myeloid_progenitor', 'cornea',
       'dendritic_cells_lymphoid_CD8a+', 'dendritic_cells_myeloid_CD8a-',
       'dendritic_plasmacytoid_B220+', 'dorsal_root_ganglia',
       'dorsal_striatum', 'epidermis', 'eyecup', 'follicular_B-cells',
       'granulo_mono_progenitor', 'granulocytes_mac1+gr1+', 'heart',
       'hippocampus', 'hypothalamus', 'intestine_large', 'intestine_small',
       'iris', 'kidney', 'lacrimal_gland', 'lens', 'liver', 'lung',
       'lymph_nodes', 'macrophage_bone_marrow_0hr',
       'macrophage_bone_marrow_24h_LPS', 'macrophage_bone_marrow_2hr_LPS',
       'macrophage_bone_marrow_6hr_LPS', 'macrophage_peri_LPS_thio_0hrs',
       'macrophage_peri_LPS_thio_1hrs', 